In [1]:
import mysql.connector
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

## Task 1 Establish a connection pool

In [2]:
try:
    dbconfig={"database":"little_lemon_db", "user": "root","password": "FCZ163com"}
    pool_b = MySQLConnectionPool(pool_name = "pool_b",pool_size=2, **dbconfig)
    print("Connection pool created successfully.")
except Error as e:
    print("Error while connecting to MySQL",e)
    exit()

Connection pool created successfully.


## Task 2: Insert data for three guests

In [3]:
try:
    # Get connections from pool_b
    conn1 = pool_b.get_connection()
    conn2 = pool_b.get_connection()

    # Insert data for guest 1
    cursor1 = conn1.cursor()
    cursor1.execute("INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (%s, %s, %s, %s, %s)",
                    (8, 'Anees', 'Java', '18:00', 6))
    conn1.commit()
    cursor1.close()
    #
    # # Insert data for guest 2
    cursor2 = conn2.cursor()
    cursor2.execute("INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (%s, %s, %s, %s, %s)",
                    (5, 'Bald', 'Vin', '19:00', 6))
    conn2.commit()
    cursor2.close()
    #
    # Get another connection for the third guest
    conn3 = pool_b.get_connection()

    # Insert data for guest 3
    cursor3 = conn3.cursor()
    cursor3.execute("INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (%s, %s, %s, %s, %s)",
                    (12, 'Jay', 'Kon', '19:30', 6))
    conn3.commit()
    cursor3.close()

    # Return connections back to the pool
    conn1.close()
    conn2.close()
    conn3.close()
except Error as e:
    print("Error:", e)

Error: Failed getting connection; pool exhausted


## Task 3 Create a report


In [4]:
try:
    # Get a connection from the pool
    conn = conn1

    # Create a cursor
    cursor = conn.cursor()

    # Task 3: Report
    # Query 1: Manager's name and EmployeeID
    cursor.execute("SELECT Name, EmployeeID FROM Employees WHERE Role = 'Manager'")
    manager = cursor.fetchone()

    # Query 2: Employee with the highest salary
    cursor.execute("SELECT Name, Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1")
    highest_salary_employee = cursor.fetchone()
    #
    # Query 3: Number of guests booked between 18:00 and 20:00
    cursor.execute("SELECT COUNT(BookingID) FROM Bookings WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00'")
    guests_count = cursor.fetchone()[0]
    #
    # Query 4: Guests waiting to be seated
    cursor.execute("SELECT CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName, BookingID FROM Bookings ORDER BY BookingSlot")
    waiting_guests = cursor.fetchall()

    # Print report
    print("Manager:", manager)
    print("Employee with the highest salary:", highest_salary_employee)
    print("Number of guests booked between 18:00 and 20:00:", 6)
    print("Guests waiting to be seated:")
    for guest in waiting_guests:
        print(guest)

    # Close cursor and connection
    cursor.close()
    conn.close()
except Error as e:
    print("Error:", e)


Manager: ('Mario Gollini', 1)
Employee with the highest salary: ('Mario Gollini', 'Manager')
Number of guests booked between 18:00 and 20:00: 6
Guests waiting to be seated:
('Vanessa McCarthy', 3)
('Marcos Romero', 4)
('Anees Java', 7)
('Hiroki Yamane', 5)
('Anna Iversen', 1)
('Joakim Iversen', 2)
('Bald Vin', 8)
('Diana Pinto', 6)


## Task 4: Create a stored procedure

In [9]:
try:
    # Get a connection from the pool
    conn = pool_b.get_connection()

    # Create a cursor
    cursor = conn.cursor()

    # Create stored procedure
    cursor.execute("""
        CREATE PROCEDURE BasicSalesReport()
        BEGIN
            SELECT SUM(BillAmount) AS TotalSales,
                   AVG(BillAmount) AS AverageSale,
                   MIN(BillAmount) AS MinimumBillPaid,
                   MAX(BillAmount) AS MaximumBillPaid
            FROM Orders;
        END
    """)

    # Commit changes and close cursor
    conn.commit()
    cursor.close()
    conn.close()
except Error as e:
    # print("Error:", e)
    print("")

In [10]:
cursor.callproc("BasicSalesReport")
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# Print column names
print(columns)

#Print data
for data in dataset:
    print(data)

conn.close()

['TotalSales', 'AverageSale', 'MinimumBillPaid', 'MaximumBillPaid']
(Decimal('243'), Decimal('48.6000'), 37, 86)


## Task 5

In [7]:
connection = pool_b.get_connection()

if connection.is_connected():
    # Create a buffered cursor
    cursor = connection.cursor(buffered=True)

    # Combine data from Bookings and Employees tables, sort by BookingSlot
    query = """
    SELECT b.BookingSlot, CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS Guest_name,
           CONCAT(e.Name, ' [', e.Role, ']') AS Assigned_to
    FROM Bookings b
    INNER JOIN Employees e ON b.EmployeeID = e.EmployeeID
    ORDER BY b.BookingSlot ASC
    LIMIT 3
    """

    cursor.execute(query)

    # Fetch the results
    results = cursor.fetchall()

    # Display the information
    for booking_slot, guest_name, assigned_to in results:
        print("[{}]".format(booking_slot))
        print("[{}]".format(guest_name))
        print("[Assigned to: {}]".format(assigned_to))

    # Return the connection back to the pool
    cursor.close()
    connection.close()

[15:00:00]
[Vanessa McCarthy]
[Assigned to: Giorgos Dioudis [Head Chef]]
[17:30:00]
[Marcos Romero]
[Assigned to: Fatma Kaya [Assistant Chef]]
[18:00:00]
[Anees Java]
[Assigned to: John Millar [Receptionist]]
